# 1️⃣  Setup ---------------------------------------------------------------

In [ ]:
import spacy
from spacy.pipeline import EntityRuler

In [ ]:
# Load the small English model
nlp = spacy.load("en_core_web_sm")

# 2️⃣  Add custom patterns for product & brand names ----------------------

In [ ]:
patterns = [
    {"label": "PRODUCT", "pattern": "Echo Dot"},
    {"label": "PRODUCT", "pattern": "Kindle Paperwhite"},
    {"label": "PRODUCT", "pattern": "Galaxy Buds"},
    {"label": "PRODUCT", "pattern": "ThinkPad X1 Carbon"},
    {"label": "ORG",      "pattern": "Amazon"},
    {"label": "ORG",      "pattern": "Samsung"},
    {"label": "ORG",      "pattern": "Lenovo"},
]

In [ ]:
ruler = EntityRuler(nlp, overwrite_ents=True)
ruler.add_patterns(patterns)
nlp.add_pipe(ruler, before="ner")

In [ ]:
# 3️⃣  Tiny rule‑based sentiment helper -----------------------------------
positive_words = {
    "excellent", "great", "love", "loved", "fantastic", "perfect",
    "amazing", "easy", "comfortable", "happy", "works", "solid"
}
negative_words = {
    "bad", "terrible", "poor", "slow", "broken", "disappointed",
    "difficult", "cheap", "hate", "heavy", "return"
}

In [ ]:
def sentiment_rule(text):
    tokens = [t.text.lower() for t in nlp.make_doc(text)]
    pos = sum(tok in positive_words for tok in tokens)
    neg = sum(tok in negative_words for tok in tokens)
    if pos == neg == 0:
        return "neutral"
    return "positive" if pos >= neg else "negative"


In [ ]:
# 4️⃣  Example reviews -----------------------------------------------------
reviews = [
    "I love my new Echo Dot! Amazon finally nailed the sound quality.",
    "The Kindle Paperwhite is great but the screen cracked after a month — very disappointed.",
    "Samsung Galaxy Buds have an amazing fit and the battery life is excellent.",
    "My ThinkPad X1 Carbon was fast at first, then turned slow and the keyboard feels cheap."
]

In [ ]:
# 5️⃣  Run the pipeline and show results ----------------------------------
for i, txt in enumerate(reviews, 1):
    doc = nlp(txt)
    ents = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ("PRODUCT", "ORG")]
    print(f"\nReview {i}: {txt}")
    print("  ➤ Entities:", ents if ents else "none found")
    print("  ➤ Sentiment:", sentiment_rule(txt))